In [23]:
# Load libraries
import numpy as np
from keras import models
from keras import layers
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
import pandas as pd
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold


In [24]:
# Importing the datasets
dataset= pd.read_csv(r'C:\Users\yosep\Desktop\ANN Keras training\Database Final 3.csv', nrows=95)
data_categories=dataset[['Duration (years)','Road Category','Type of Network Intervention','Number of Province','Geographical Profile','EHS/GHS','GDP','3 years Average GDP Growth','RDP','Change of Power','Ideoogy ','Population','c/km2']]
data_encoded=pd.get_dummies(data_categories)
print(data_encoded.columns)

data_encoded.head()

Index(['Duration (years)', 'Number of Province', 'GDP',
       '3 years Average GDP Growth', 'RDP', 'Population', 'c/km2',
       'Road Category_National', 'Road Category_Provincial',
       'Type of Network Intervention_Modification',
       'Type of Network Intervention_New', 'Geographical Profile_Combination',
       'Geographical Profile_Rural', 'Geographical Profile_Urban',
       'EHS/GHS_No', 'EHS/GHS_Yes', 'Change of Power_No',
       'Change of Power_Yes', 'Ideoogy _Center', 'Ideoogy _Center-Left',
       'Ideoogy _Center-Right ', 'Ideoogy _CenterLeft'],
      dtype='object')


,Duration (years),Number of Province,GDP,3 years Average GDP Growth,RDP,Population,c/km2,Road Category_National,Road Category_Provincial,Type of Network Intervention_Modification,...,Geographical Profile_Rural,Geographical Profile_Urban,EHS/GHS_No,EHS/GHS_Yes,Change of Power_No,Change of Power_Yes,Ideoogy _Center,Ideoogy _Center-Left,Ideoogy _Center-Right,Ideoogy _CenterLeft
0,6.67,2,414838,0.053704,0.2715,3601880,20357.332510,1,0,1,...,1,0,0,1,1,0,0,0,0,1
1,5.04,1,545609,0.033351,0.0612,1136695,6868.199366,1,0,1,...,1,0,1,0,1,0,1,0,0,0
2,2.95,1,523939,0.032014,0.1864,2587265,11389.300570,1,0,0,...,0,0,0,1,1,0,1,0,0,0
3,4.38,1,364961,0.057198,0.0826,1079428,9363.410326,1,0,1,...,1,0,0,1,0,1,0,0,0,1
4,2.42,1,639163,0.054203,0.0844,1201350,10836.177000,1,0,1,...,0,0,0,1,0,1,0,0,1,0


In [45]:
#Identify input features and output (X and y)
X = data_encoded.drop(['Duration (years)'], axis=1)
y = data_encoded.iloc[:, 0].values
y = np.array(y).reshape((-1, 1))

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.1, random_state=0)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler(1)
features_train= sc.fit_transform(X_train)
features_test= sc.fit_transform(X_test)
target_train= sc.fit_transform(y_train)
target_test= sc.fit_transform(y_test)

C:\Users\yosep\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\yosep\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\yosep\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\yosep\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [46]:
# Create function returning a compiled network
def create_network():
    
    # Start neural network
    network = models.Sequential()

    # Add fully connected layer with a ReLU activation function
    network.add(layers.Dense(units=11, kernel_initializer='normal',activation='relu', input_shape=(21,)))
    
    # Add fully connected layer with a ReLU activation function
    network.add(layers.Dense(units=11, kernel_initializer='normal',activation='relu'))

    # Add fully connected layer without activation functon
    network.add(layers.Dense(units=1, kernel_initializer='normal'))

    # Compile neural network
    network.compile(loss='mean_squared_error', # Regression Problem
                    optimizer='adam',) # Adam
    
    # Return compiled network
    return network

In [47]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=create_network, epochs=100, batch_size=5, verbose=0)

In [49]:
# Evaluate neural network using three-fold cross-validation
kfold = KFold(n_splits=5, random_state=seed)
results = cross_val_score(estimator, features_train, target_train, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Results: -1.39 (0.48) MSE


In [50]:
estimator.fit(features_train,target_train)

In [52]:
prediction=estimator.predict(features_test)

In [53]:
prediction

array([ 0.374793  ,  0.67883486, -1.0177767 , -0.32220381, -0.16510625,
       -0.45045555,  0.41941306,  0.2629265 ,  0.3525635 , -0.35914278],
      dtype=float32)

In [58]:
print(target_test)

[[-0.6582452 ]
 [ 1.59425237]
 [-1.76837182]
 [ 0.30447871]
 [ 0.2492027 ]
 [ 0.74668682]
 [ 0.16168234]
 [-0.83328591]
 [ 1.20732027]
 [-1.00372029]]
